In [1]:
import os
import shutil
from PIL import Image, UnidentifiedImageError
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint

# ─── 1) Directory setup ────────────────────────────────────────────────────────
BASE_DIR   = r'C:\Users\ADMIN\OneDrive\Desktop\AI_PROJECT'
TRAIN_DIR  = os.path.join(BASE_DIR, 'train')
TEST_DIR   = os.path.join(BASE_DIR, 'test')

# ─── 2) Clean out corrupt images ───────────────────────────────────────────────
def clean_bad_images(dir_path, bad_root):
    """
    Move any file that PIL cannot open into bad_root/<train|test>/<class_name>/.
    """
    for class_name in os.listdir(dir_path):
        class_dir = os.path.join(dir_path, class_name)
        if not os.path.isdir(class_dir):
            continue
        for fname in os.listdir(class_dir):
            fpath = os.path.join(class_dir, fname)
            if not os.path.isfile(fpath):
                continue
            try:
                with Image.open(fpath) as img:
                    img.verify()
            except (UnidentifiedImageError, OSError):
                bad_dir = os.path.join(bad_root, os.path.basename(dir_path), class_name)
                os.makedirs(bad_dir, exist_ok=True)
                print(f"Moving corrupt file: {fpath}")
                shutil.move(fpath, os.path.join(bad_dir, fname))

bad_root = os.path.join(BASE_DIR, 'bad_data')
clean_bad_images(TRAIN_DIR, bad_root)
clean_bad_images(TEST_DIR, bad_root)


# ─── 3) Hyperparameters ────────────────────────────────────────────────────────
IMG_SIZE   = (150, 150)
BATCH_SIZE = 16
EPOCHS     = 15

# ─── 4) Data generators ────────────────────────────────────────────────────────
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.1
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'
)
test_gen = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)


# ─── 5) Build the CNN ───────────────────────────────────────────────────────────
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(*IMG_SIZE, 3)),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)
model.summary()


# ─── 6) Train & checkpoint ─────────────────────────────────────────────────────
checkpoint = ModelCheckpoint(
    'rubbish_cnn_best.h5',
    save_best_only=True,
    monitor='val_accuracy',
    mode='max'
)

history = model.fit(
    train_gen,
    epochs=EPOCHS,
    validation_data=test_gen,
    callbacks=[checkpoint]
)


# ─── 7) Load best weights & evaluate ───────────────────────────────────────────
model.load_weights('rubbish_cnn_best.h5')
loss, acc = model.evaluate(test_gen, verbose=1)
print(f"\nTest accuracy on roadside rubbish detection: {acc * 100:.2f}%")


Moving corrupt file: C:\Users\ADMIN\OneDrive\Desktop\AI_PROJECT\train\RUBBISH\1c269143-b0b4-457d-b85e-e7e6e61d0817.jpeg
Moving corrupt file: C:\Users\ADMIN\OneDrive\Desktop\AI_PROJECT\test\RUBBISH\1c269143-b0b4-457d-b85e-e7e6e61d0817.jpeg
Found 87 images belonging to 2 classes.
Found 87 images belonging to 2 classes.


C:\Users\ADMIN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 36992)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     4,735,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,828,481 (18.42 MB)

...

C:\Users\ADMIN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 265ms/step - accuracy: 0.5086 - loss: 0.8680

6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 661ms/step - accuracy: 0.5131 - loss: 0.8645 - val_accuracy: 0.7011 - val_loss: 0.6295
Epoch 2/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 474ms/step - accuracy: 0.6509 - loss: 0.6646 - val_accuracy: 0.6897 - val_loss: 0.5585
Epoch 3/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 264ms/step - accuracy: 0.7504 - loss: 0.5782

6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 487ms/step - accuracy: 0.7499 - loss: 0.5788 - val_accuracy: 0.9655 - val_loss: 0.3346
Epoch 4/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 463ms/step - accuracy: 0.8594 - loss: 0.4202 - val_accuracy: 0.9655 - val_loss: 0.1674
Epoch 5/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 468ms/step - accuracy: 0.8743 - loss: 0.3162 - val_accuracy: 0.9655 - val_loss: 0.1359
Epoch 6/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 465ms/step - accuracy: 0.9630 - loss: 0.1417 - val_accuracy: 0.9540 - val_loss: 0.1996
Epoch 7/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 492ms/step - accuracy: 0.8997 - loss: 0.3728 - val_accuracy: 0.9425 - val_loss: 0.1587
Epoch 8/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 476ms/step - accuracy: 0.8441 - loss: 0.4092 - val_accuracy: 0.8851 - val_loss: 0.3395
Epoch 9/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 273ms/step - accuracy: 0.9291 - loss: 0.3317

6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 480ms/step - accuracy: 0.9294 - loss: 0.3314 - val_accuracy: 0.9770 - val_loss: 0.0833
Epoch 10/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 474ms/step - accuracy: 0.9038 - loss: 0.3171 - val_accuracy: 0.9770 - val_loss: 0.1505
Epoch 11/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 442ms/step - accuracy: 0.9527 - loss: 0.1534 - val_accuracy: 0.9655 - val_loss: 0.1598
Epoch 12/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 453ms/step - accuracy: 0.9487 - loss: 0.2017 - val_accuracy: 0.9770 - val_loss: 0.0825
Epoch 13/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 494ms/step - accuracy: 0.9200 - loss: 0.3370 - val_accuracy: 0.9655 - val_loss: 0.1342
Epoch 14/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 458ms/step - accuracy: 0.9301 - loss: 0.1716 - val_accuracy: 0.9770 - val_loss: 0.0978
Epoch 15/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 501ms/step - accuracy: 0.9610 - loss: 0.1630 - val_accuracy: 0.9655 - val_loss: 0.0905
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 149ms/step - accuracy: 0.9705 - loss: 0.0804

Test accuracy on roadside rubbish detection: 97.7